In [11]:
import cv2
import os
import pandas as pd
import numpy as np

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    print(f"Extracted {len(frames)} frames at {fps} FPS")
    return frames, fps

def detect_cricket_ball(frames):
    annotations = []
    trajectory = []
    prev_gray = None

    for frame_id, frame in enumerate(frames):
        h, w = frame.shape[:2]
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Color ranges (red + white)
        lower_red1 = np.array([0, 60, 60])
        upper_red1 = np.array([12, 255, 255])
        lower_red2 = np.array([168, 60, 60])
        upper_red2 = np.array([180, 255, 255])
        lower_white = np.array([0, 0, 210])
        upper_white = np.array([180, 30, 255])

        mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask_white = cv2.inRange(hsv, lower_white, upper_white)

        mask = cv2.bitwise_or(mask_red1, mask_red2)
        mask = cv2.bitwise_or(mask, mask_white)

        # Clean mask
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        contours, _ = cv2.findContours(
            mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        visible = 0
        x, y = -1, -1
        candidates = []

        for contour in contours:
            area = cv2.contourArea(contour)

            if 40 < area < 500:
                perimeter = cv2.arcLength(contour, True)
                if perimeter == 0:
                    continue

                circularity = 4 * np.pi * area / (perimeter * perimeter)

                if circularity > 0.7:
                    bx, by, bw, bh = cv2.boundingRect(contour)

                    aspect_ratio = bw / float(bh)
                    if aspect_ratio < 0.85 or aspect_ratio > 1.15:
                        continue

                    cx = bx + bw // 2
                    cy = by + bh // 2

                    # Ignore bottom & extreme top
                    if cy > h * 0.85 or cy < h * 0.25:
                        continue

                    dist = float("inf")
                    if trajectory:
                        px, py = trajectory[-1]
                        dist = np.sqrt((cx - px) ** 2 + (cy - py) ** 2)

                    score = circularity * area
                    candidates.append((cx, cy, score, dist))

        # Choose best candidate
        if candidates:
            candidates.sort(key=lambda c: (-c[2], c[3]))
            best_cx, best_cy, best_score, best_dist = candidates[0]

            if not trajectory:
                x, y = best_cx, best_cy
                visible = 1
                trajectory.append((x, y))
            else:
                if 8 < best_dist < 70:
                    x, y = best_cx, best_cy
                    visible = 1
                    trajectory.append((x, y))

        if len(trajectory) > 1:
            pts = np.array(trajectory, np.int32)
            cv2.polylines(frame, [pts], False, (255, 0, 255), 4)

        if visible:
            cv2.circle(frame, (x, y), 12, (0, 255, 0), -1)
            cv2.putText(
                frame,
                f"BALL({x},{y})",
                (x + 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 0, 255),
                2,
            )

        annotations.append([frame_id, x, y, visible])
        frames[frame_id] = frame

    return frames, annotations

def save_video(frames, fps, output_path="output_with_trajectory_1.mp4"):
    h, w = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    for frame in frames:
        out.write(frame)

    out.release()
    print(f"Video saved: {output_path}")

def save_annotations(annotations, filename="ball_annotations_1.csv"):
    df = pd.DataFrame(
        annotations,
        columns=["frame_index", "x_centroid", "y_centroid", "visibility_flag"],
    )
    df.to_csv(filename, index=False)
    print(f"Annotation file saved: {filename}")

if __name__ == "__main__":

    video_path = "1.mp4"

    frames, fps = extract_frames(video_path)
    processed_frames, annotations = detect_cricket_ball(frames)

    save_video(processed_frames, fps)
    save_annotations(annotations)

Extracted 31 frames at 25 FPS
Video saved: output_with_trajectory_1.mp4
Annotation file saved: ball_annotations_1.csv
